In [1]:
%autosave 0
%reload_ext autoreload
%autoreload 2
%load_ext line_profiler

Autosave disabled


In [2]:
from multiprocessing import Pool

import ipdb

import cv2
import nibabel as nib
from skimage import measure
from skimage import morphology

from fastai.vision import *

/home/majiechao/anaconda3/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [3]:
with open('../shared/voc_cmap.pkl','rb') as f:
    cmap_list = pickle.load(f)
    cmap = np.array(cmap_list).reshape(-1,3)

FileNotFoundError: [Errno 2] No such file or directory: '../shared/voc_cmap.pkl'

### make/copy/link

In [3]:
src_root = Path('../../../../pseudo/r4')
ds_root = Path('origin/xinan')

for ds_name,src in (
    ('xnb2','/lung_general_data/Datasets/xinan/xnky_batch2/CT20200319003'),
    ('xn3','/lung_general_data/Datasets/COV_DATA/xinan/CT20200217001'),
#     ('xn4','/lung_general_data/Datasets/COV_DATA/xinan/CT20200220002'),
#     ('hn1','/lung_general_data/Datasets/COV_DATA/hndata/CT20200225001'),
#     ('hn2','/lung_general_data/Datasets/COV_DATA/hndata/CT20200226001'),
#     ('hn3','/lung_general_data/Datasets/COV_DATA/hndata/CT20200226002'),
#     ('hn4','/lung_general_data/Datasets/COV_DATA/hndata/CT20200306001'),
#     ('hn4t','/lung_general_data/Datasets/COV_DATA/hndata/CT20200308001'),
#     ('gz','/lung_general_data/Datasets/COV_DATA/zzdata/CT20200219001'),
):
    sub_dir1 = ds_root/ds_name/'work_data/lung_det_xinan_keyan'
    sub_dir1.mkdir(parents=True, exist_ok=True)
    im_dir = sub_dir1/'3d_slice_origin'
    if not im_dir.exists():
        im_dir.symlink_to(src_root/ds_name)
    sub_dir2 = sub_dir1/'config/coco_json_origin'
    sub_dir2.mkdir(parents=True, exist_ok=True)
    src_fp = Path(src)/'work_data/lung_det_xinan_keyan/config/coco_json_origin/image_list_1221.txt'
    cmd = f'rsync -au i13:{src_fp} {sub_dir2}'
    print(cmd)
    os.system(cmd)

FileExistsError: [Errno 17] File exists: '../../../../pseudo/r4/xnb2' -> 'origin/xinan/xnb2/work_data/lung_det_xinan_keyan/3d_slice_origin'

In [5]:
pwd

'/home/majiechao/fai_classification/diag/data/lung_inflammation'

#### negative slices

In [ ]:
ds_root = Path('origin/xinan')
im_dir = ds_root/'neg/work_data/lung_det_xinan_keyan/3d_slice_origin'
im_dir.mkdir(parents=True, exist_ok=True)
list_dir = ds_root/'neg/work_data/lung_det_xinan_keyan/config/coco_json_origin'
list_dir.mkdir(parents=True, exist_ok=True)

In [190]:
with open(list_dir/'nodule_lesion_random_0611_train_image_list.txt','r') as f:
    fp_list = [l.strip() for l in f.readlines()]

In [ ]:
fp_list[:2]

In [206]:
src_dir = Path('/lung_general_data/lung_det/lung_nodule_data/lung_nodule_0213/3d_slice_origin')
dst_dir = im_dir

for fp in fp_list:
    fp = Path(fp)
    i = int(fp.stem)
    for j in (i-1,i,i+1):
        if i<0:
            continue
        im_fp = fp.parent/f'{j:03}.png'
        dst_fp = dst_dir/im_fp
        dst_fp.parent.mkdir(parents=True, exist_ok=True)
        cmd = f'rsync -au i13:{src_dir/im_fp} {dst_fp}'
        #print(cmd)
        os.system(cmd)

### prepare images

In [106]:
cache_root = Path('a1')/'diag'
(cache_root/'image').mkdir(parents=True)

ct_list = sorted(Path('origin/xinan').glob('*/work_data/lung_det_xinan_keyan/3d_slice_origin/*/*/*/'))

for ctid,ct_fp in enumerate(ct_list):
    png_list = [t for t in sorted(list(ct_fp.glob('*.png'))) if '-' not in str(t.stem)]
    for k,png_fp in enumerate(png_list):
        # link image
        from_fp =cache_root/f'image/{ctid:06}_{k:04}.png'
        if not from_fp.exists():
            from_fp.symlink_to(f'../../../{png_fp}')

### make list day1

In [44]:
# find pids
all_pos_fps = {}
all_pos_pids = {}
all_fps = {}
all_pids = {}
for subset in ['cov','xnky_batch0','xnky_batch1']:
    with open(f'origin/xinan/{subset}/work_data/lung_det_xinan_keyan/config/coco_json_origin/image_list_1221.txt','r') as f:
        all_pos_fps[subset] = sorted([l.strip() for l in f.readlines()])
        all_pos_pids[subset] = set([fp.split('/')[0] for fp in all_pos_fps[subset]])
    all_fps[subset] = sorted(Path(f'origin/xinan/{subset}/work_data/lung_det_xinan_keyan/3d_slice_origin').glob('*/*/*/*.png'))
    all_pids[subset] = set([fp.parts[-4] for fp in all_fps[subset]])

vl_pids = {k:np.random.choice(list(v), 10, replace=False).tolist() for k,v in all_pos_pids.items()}

In [62]:
# bacterial:2; fungal:3;
cls_map = {'0':'2','1':'3'}
pid2cls = {}
for subset in ['xnky_batch0','xnky_batch1']:
    with open(f'origin/xinan/{subset}/auto_rebuild/pid_list.txt','r') as f:
        for l in f.readlines():
            fp,cls = l.strip().split()
            pid2cls[fp.split('_')[0]] = cls_map[cls]

In [83]:
# bg:0; cov:1; bacterial:2; fungal:3; non-co-virus:4
ds_name = 'diag'

ct_list = sorted(Path('origin/xinan').glob('*/work_data/lung_det_xinan_keyan/3d_slice_origin/*/*/*/'))

records = []
for ctid,ct_fp in enumerate(ct_list):
    if 'cov'==ct_fp.parts[2]:
        mask = '1'
    else:
        mask = pid2cls[ct_fp.parts[-3]]
    image_from = Path(f'{ds_name}/image/{ctid:06}_')
    split_idx = 0
    for k,v in vl_pids.items():
        if ct_fp.parts[-3] in v:
            split_idx = 100
            break
    png_list = [t for t in sorted(list(ct_fp.glob('*.png'))) if '-' not in str(t.stem)]
    h,w,d = None,None,len(png_list)
    for idx,png_fp in enumerate(png_list):
        if h is None:
            w,h = PIL.Image.open(png_fp).size
        if '/'.join(png_fp.parts[-4:]) in all_pos_fps[ct_fp.parts[2]]:
            mask += mask[0]
        else:
            mask += '0'
    records.append(['',mask,'',h,w,d,'det',image_from,'','',str(split_idx)])

In [81]:
columns = ['image','mask','instance','h','w','d','src','image_from','mask_from','instance_from','split']

In [85]:
df = pd.DataFrame(records, columns=columns)
print([(df.split==str(i)).sum() for i in (0,100)])

df.loc[~((df.h==df.w)&(df.h==512)),'split'] = '300'
[(df.split==str(i)).sum() for i in (0,100)]

[245, 37]


[245, 37]

In [88]:
df.to_csv('dd1.csv', index=False)

### make list day2

In [8]:
# find pids
all_pos_fps = {}
all_pos_pids = {}
all_fps = {}
all_pids = {}
for subset in ('cov','normal_virus','xnky_batch0','xnky_batch1'):
    with open(f'origin/xinan/{subset}/work_data/lung_det_xinan_keyan/config/coco_json_origin/image_list_1221.txt','r') as f:
        all_pos_fps[subset] = sorted([l.strip() for l in f.readlines()])
        all_pos_pids[subset] = set([fp.split('/')[0] for fp in all_pos_fps[subset]])
    all_fps[subset] = sorted(Path(f'origin/xinan/{subset}/work_data/lung_det_xinan_keyan/3d_slice_origin').glob('*/*/*/*.png'))
    all_pids[subset] = set([fp.parts[-4] for fp in all_fps[subset]])

vl_pids = {k:np.random.choice(list(v), 10, replace=False).tolist() for k,v in all_pos_pids.items()}

FileNotFoundError: [Errno 2] No such file or directory: 'origin/xinan/cov/work_data/lung_det_xinan_keyan/config/coco_json_origin/image_list_1221.txt'

In [108]:
# bacterial:2; fungal:3;
cls_map = {'0':'2','1':'3'}
pid2cls = {}
for subset in ['xnky_batch0','xnky_batch1']:
    with open(f'origin/xinan/{subset}/auto_rebuild/pid_list.txt','r') as f:
        for l in f.readlines():
            fp,cls = l.strip().split()
            pid2cls[fp.split('_')[0]] = cls_map[cls]

In [109]:
# bg:0; cov:1; bacterial:2; fungal:3; non-co-virus:4
ds_name = 'diag'

ct_list = sorted(Path('origin/xinan').glob('*/work_data/lung_det_xinan_keyan/3d_slice_origin/*/*/*/'))

records = []
for ctid,ct_fp in enumerate(ct_list):
    if 'cov'==ct_fp.parts[2]:
        mask = '1'
    elif 'normal_virus'==ct_fp.parts[2]:
        mask = '4'
    else:
        mask = pid2cls[ct_fp.parts[-3]]
    image_from = Path(f'{ds_name}/image/{ctid:06}_')
    split_idx = 0
    for k,v in vl_pids.items():
        if ct_fp.parts[-3] in v:
            split_idx = 100
            break
    png_list = [t for t in sorted(list(ct_fp.glob('*.png'))) if '-' not in str(t.stem)]
    h,w,d = None,None,len(png_list)
    for idx,png_fp in enumerate(png_list):
        if h is None:
            w,h = PIL.Image.open(png_fp).size
        if '/'.join(png_fp.parts[-4:]) in all_pos_fps[ct_fp.parts[2]]:
            mask += mask[0]
        else:
            mask += '0'
    records.append(['',mask,'',h,w,d,'det',image_from,'','',str(split_idx)])

In [110]:
columns = ['image','mask','instance','h','w','d','src','image_from','mask_from','instance_from','split']

In [111]:
df = pd.DataFrame(records, columns=columns)
print([(df.split==str(i)).sum() for i in (0,100)])

df.loc[~((df.h==df.w)&(df.h==512)),'split'] = '300'
[(df.split==str(i)).sum() for i in (0,100)]

[761, 67]


[761, 67]

In [112]:
df.to_csv('dd2.csv', index=False)

In [117]:
df.head()

,image,mask,instance,h,w,d,src,image_from,mask_from,instance_from,split
0,,1000000000000000000000000000000000000000000000...,,512,512,251,det,diag/image/000000_,,,0
1,,1000000000000000000000000000000000000000000000...,,512,512,272,det,diag/image/000001_,,,0
2,,1000000000000000000000000000000000000000000000...,,512,512,134,det,diag/image/000002_,,,0
3,,1000000000000000000000000000000000000000000111...,,512,512,175,det,diag/image/000003_,,,0
4,,1000000000000000000000000000000000111111111110...,,512,512,168,det,diag/image/000004_,,,0


In [182]:
df.d.sum()

55178

In [119]:
for i in range(1,5):
    print(sum([t[0]==f'{i}' for t in df['mask']]))

199
43
40
546


### make list hainan(by maoli)

In [161]:
# xlsx to csv
tdf = pd.read_excel('origin/hainan/gt.xlsx')
tdf.to_csv('origin/hainan/gt.csv', index=False)

In [167]:
cache_root = Path('a1')/'diag_hainan'
(cache_root/'image').mkdir(parents=True)

ct_list = sorted(Path('origin/hainan/png').glob('*/'))

for ctid,ct_fp in enumerate(ct_list):
    png_list = [t for t in sorted(list(ct_fp.glob('*.png'))) if '-' not in str(t.stem)]
    for k,png_fp in enumerate(png_list):
        # link image
        from_fp =cache_root/f'image/{ctid:06}_{k:04}.png'
        if not from_fp.exists():
            from_fp.symlink_to(f'../../../{png_fp}')

In [179]:
# now: cov:1; non-co-virus:0
#  to: bg:0; cov:1; bacterial:2; fungal:3; non-co-virus:4

tdf = pd.read_csv('origin/hainan/gt.csv')

records = []
for ctid,ct_fp in enumerate(ct_list):
    pid,date = ct_fp.name.split('_')
    image_from = Path(f'diag_hainan/image/{ctid:06}_')
    png_list = [t for t in sorted(list(ct_fp.glob('*.png'))) if '-' not in str(t.stem)]
    d = len(png_list)
    w,h = PIL.Image.open(png_list[0]).size
    found = (tdf.pid==int(pid))&(tdf.date==int(date))
    if found.sum()>0:
        label = tdf.loc[found].label.values[0]
    else:
        assert ct_fp.name=='0826166_20200127'
        label = 1
    if label==0:
        label = 4
    mask = f'{label}' * (d+1)
    records.append(['',mask,'',h,w,d,'hainan',image_from,'','','0'])

In [180]:
columns = ['image','mask','instance','h','w','d','src','image_from','mask_from','instance_from','split']

In [181]:
df = pd.DataFrame(records, columns=columns)
df.to_csv('dd_hainan.csv', index=False)

In [178]:
df.head()

,image,mask,instance,h,w,d,src,image_from,mask_from,instance_from,split
0,,1111111111111111111111111111111111111111111111...,,512,512,201,hainan,diag_hainan/image/000000_,,,0
1,,1111111111111111111111111111111111111111111111...,,512,512,207,hainan,diag_hainan/image/000001_,,,0
2,,1111111111111111111111111111111111111111111111...,,512,512,209,hainan,diag_hainan/image/000002_,,,0
3,,1111111111111111111111111111111111111111111111...,,512,512,217,hainan,diag_hainan/image/000003_,,,0
4,,1111111111111111111111111111111111111111111111...,,512,512,184,hainan,diag_hainan/image/000004_,,,0
